## Julia on Colab Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools https://github.com/Dale-Black/DistanceTransforms.jl.git"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=1
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.1 on the current Colab Runtime...
2021-06-22 23:51:05 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz [112784227/112784227] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package https://github.com/Dale-Black/DistanceTransforms.jl.git...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.1
Commit 6aaedecc44 (2021-04-23 05:59 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver2)
Environment:
  JULIA_NUM_THREADS = 1


## Import Packages

In [2]:
using BenchmarkTools
using DistanceTransforms

## Load test data

In [4]:
x = rand([0,1], (112, 112, 96, 2, 1))
y = rand([0,1], (112, 112, 96, 2, 1))

x_dtm = DistanceTransforms.compute_dtm(x)
y_dtm = DistanceTransforms.compute_dtm(y);

## Vanilla dice loss

In [5]:
@benchmark DistanceTransforms.dice_loss($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  55.13 MiB
  allocs estimate:  9
  --------------
  minimum time:     11.093 ms (0.00% GC)
  median time:      13.144 ms (12.30% GC)
  mean time:        12.996 ms (9.66% GC)
  maximum time:     16.505 ms (25.68% GC)
  --------------
  samples:          384
  evals/sample:     1

## Vanilla Hausdorff loss

In [6]:
@benchmark DistanceTransforms.hd_loss($x, $y, $x_dtm, $y_dtm)

BenchmarkTools.Trial: 
  memory estimate:  18.38 MiB
  allocs estimate:  7
  --------------
  minimum time:     7.078 ms (0.00% GC)
  median time:      7.678 ms (0.00% GC)
  mean time:        8.152 ms (6.11% GC)
  maximum time:     11.066 ms (27.45% GC)
  --------------
  samples:          612
  evals/sample:     1

In [7]:
@benchmark DistanceTransforms.hd_loss($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))

BenchmarkTools.Trial: 
  memory estimate:  1.36 GiB
  allocs estimate:  24328272
  --------------
  minimum time:     1.876 s (10.64% GC)
  median time:      1.887 s (10.80% GC)
  mean time:        1.893 s (10.85% GC)
  maximum time:     1.917 s (11.10% GC)
  --------------
  samples:          3
  evals/sample:     1

## Parallelized dice loss

In [8]:
@benchmark DistanceTransforms.dice_lossP($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  2
  --------------
  minimum time:     6.872 ms (0.00% GC)
  median time:      7.078 ms (0.00% GC)
  mean time:        7.085 ms (0.00% GC)
  maximum time:     7.866 ms (0.00% GC)
  --------------
  samples:          704
  evals/sample:     1

## Parallelized Hausdorff loss

In [9]:
@benchmark DistanceTransforms.hd_lossP($x, $y, $x_dtm, $y_dtm)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     3.295 ms (0.00% GC)
  median time:      3.517 ms (0.00% GC)
  mean time:        3.525 ms (0.00% GC)
  maximum time:     4.110 ms (0.00% GC)
  --------------
  samples:          1409
  evals/sample:     1

In [10]:
@benchmark DistanceTransforms.hd_lossP($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))

BenchmarkTools.Trial: 
  memory estimate:  1.34 GiB
  allocs estimate:  24328266
  --------------
  minimum time:     1.860 s (10.75% GC)
  median time:      1.890 s (10.96% GC)
  mean time:        1.884 s (10.88% GC)
  maximum time:     1.901 s (10.93% GC)
  --------------
  samples:          3
  evals/sample:     1